# Name: Nhat Minh Dang
# ID: 222172836
# SIT 378 

# Course Detection

## 1) Imported library:

In [3]:
import os
import librosa
import numpy as np
import time 

# 2) Configuration:

In [5]:
AUDIO_PATH = "Data" 
MIN_FREQ = 300  
MAX_FREQ = 4000  
MIN_DURATION = 1  
ANIMAL_THRESHOLD = 0.2 
NUM_RUNS = 1
TOP_DB = 20

# 3) Count the number of files:
This function counts the number of files to verify valid imput.

In [7]:
def count_files_in_directory_and_subdirectories(directory_path):
    try:
        total_files = 0
        for root, _, files in os.walk(directory_path):
            total_files += len(files) 
        return total_files
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return 0
    except PermissionError:
        print(f"Permission denied to access: {directory_path}")
        return 0

file_count = count_files_in_directory_and_subdirectories(AUDIO_PATH)
print(f"Total number of files in '{AUDIO_PATH}' and its subdirectories: {file_count}")


Total number of files in 'Data' and its subdirectories: 7161


In [8]:
# def count_files_in_directory(directory_path):
#     try:
#         all_items = os.listdir(directory_path)
#         files = [item for item in all_items if os.path.isfile(os.path.join(directory_path, item))]
#         return len(files)
#     except FileNotFoundError:
#         print(f"Directory not found: {directory_path}")
#         return 0
#     except PermissionError:
#         print(f"Permission denied to access: {directory_path}")
#         return 0

# file_count = count_files_in_directory(AUDIO_PATH)
# print(f"Number of files in '{AUDIO_PATH}': {file_count}")


# 4) Main functions:
This is the main processing steps for Animal Detection. These includes:
- Import audio files with original sample rate and perform Trimming Silence.
- Extract features and use FFT to determine whether the sound is animal or not.
- Record the list of files, executing multiple times and display the accuracy and average execution time.

In [10]:
# def list_audio_files(audio_path):
#     return [os.path.join(audio_path, f) for f in os.listdir(audio_path) if f.endswith('.wav')]

def list_audio_files(audio_path):
    audio_files = []
    # Traverse the directory and its subdirectories
    for root, _, files in os.walk(audio_path):
        for file in files:
            if file.endswith('.wav') or file.endswith('.mp3'):  
                audio_files.append(os.path.join(root, file))
    return audio_files

def load_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)  
        y_trimmed, _ = librosa.effects.trim(y, top_db=TOP_DB)
        return y_trimmed, sr
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None, None

def extract_features(y, sr):
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < MIN_DURATION:
        return None  
    fft = np.abs(np.fft.fft(y))  
    freqs = np.fft.fftfreq(len(fft), d=1/sr)  
    return freqs, fft

def is_animal_sound(freqs, fft):
    valid_indices = (freqs > 0) & (freqs >= MIN_FREQ) & (freqs <= MAX_FREQ)  
    energy_in_range = np.sum(fft[valid_indices]**2)  
    total_energy = np.sum(fft**2)  
    if total_energy == 0:  
        return False
    proportion = energy_in_range / total_energy
    return proportion >= ANIMAL_THRESHOLD

def detect_animal_sounds(audio_path):
    audio_files = list_audio_files(audio_path)
    if not audio_files:
        print("No audio files found in the directory.")
        return [], 0.0

    animal_files = []
    for file in audio_files:
        y, sr = load_audio(file)
        if y is None or sr is None:
            continue
        features = extract_features(y, sr)
        if features is None:
            continue
        freqs, fft = features
        if is_animal_sound(freqs, fft):
            animal_files.append(file)

    accuracy = len(animal_files) / len(audio_files)
    return animal_files, accuracy

# 5) Execution codes:
These are the codes that executes all functions above and display the result.

In [12]:
execution_times = []

for run in range(NUM_RUNS):
    start_time = time.time()  
    
    animal_files, accuracy = detect_animal_sounds(AUDIO_PATH)
    
    end_time = time.time()  
    execution_time = end_time - start_time  
    execution_times.append(execution_time)
    
    print(f"Run {run + 1}:")
    print(f"Animal sound files: {animal_files[:10]}")
    print(f"Detection Accuracy: {accuracy * 100:.2f}%")
    print(f"Execution Time: {execution_time:.2f} seconds\n")

average_time = np.mean(execution_times)

print("Execution Times:")
print("-----------------")
for i, exec_time in enumerate(execution_times, 1):
    print(f"Run {i}: {exec_time:.2f} seconds")
print("-----------------")
print(f"Average Execution Time: {average_time:.2f} seconds")

Run 1:
Animal sound files: ['Data\\Acanthiza chrysorrhoa\\region_55.750-57.750.mp3', 'Data\\Acanthiza chrysorrhoa\\region_61.750-63.750.mp3', 'Data\\Acanthiza chrysorrhoa\\region_74.000-76.000.mp3', 'Data\\Acanthiza chrysorrhoa\\region_84.500-86.500.mp3', 'Data\\Acanthiza lineata\\region_0.700-2.700.mp3', 'Data\\Acanthiza lineata\\region_34.450-36.450.mp3', 'Data\\Acanthiza lineata\\region_38.450-40.450.mp3', 'Data\\Acanthiza lineata\\region_40.450-42.450.mp3', 'Data\\Acanthiza lineata\\region_48.350-50.200.mp3', 'Data\\Acanthiza lineata\\region_5.200-7.200.mp3']
Detection Accuracy: 44.76%
Execution Time: 187.13 seconds

Execution Times:
-----------------
Run 1: 187.13 seconds
-----------------
Average Execution Time: 187.13 seconds
